<a href="https://colab.research.google.com/github/GeoKauko/TheNavySeals/blob/main/3_Postprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Postprocessing



In [1]:
#Connect to the google drive if you use Google Colaboratory
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Install libraries

In [4]:
! pip install torchvision rasterio numpy -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 45.2 MB/s eta 0:00:00


#### Import libraries

In [5]:
import os
import torch
from torchvision import transforms
import numpy as np
from PIL import Image
import rasterio

## Load the whole satellite image

In [8]:
# Define File directories, this is just one image. We can make a for loop to run all images
data_folder = "/content/drive/MyDrive/SealNN/Data/Maxar"
img_name = '22MAR25134903-P3DS-014983717010_01_P001'
image_dir = os.path.join(data_folder, img_name + '.TIF')
print(image_dir)

/content/drive/MyDrive/SealNN/Data/Maxar/22MAR25134903-P3DS-014983717010_01_P001.TIF


In [9]:
# Open the image
src = rasterio.open(image_dir)
print(src.meta['transform'])

| 0.30, 0.00, 397027.20|
| 0.00,-0.30, 7973998.80|
| 0.00, 0.00, 1.00|


In [10]:
# Read image data
image_data = src.read()
# Convert to torch tensor
image_tensor = torch.tensor(image_data, dtype=torch.float32)
# Permute dimensions to have channels last
image_tensor

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [11]:
image_tensor = image_tensor.permute(1, 2, 0)  # (bands, height, width) to (height, width, bands)
print(image_tensor.shape)

torch.Size([34924, 35020, 1])


In [12]:
# Function to create patches
def create_patches(image_tensor, patch_size):
    patches = []
    _, height, width = image_tensor.shape
    for i in range(0, height, patch_size):
        for j in range(0, width, patch_size):
            patch = image_tensor[i:i + patch_size, j:j + patch_size, :]
            if patch.shape[0] == patch_size and patch.shape[1] == patch_size:
                patches.append(patch)
    return torch.stack(patches)

In [13]:
patchsize = 336
patches_tensor = create_patches(image_tensor, patchsize)
print(f"There are {patches_tensor.shape[0]} number of training patches")

There are 103 number of training patches


In [14]:
# Save patches
output_path = os.path.join(data_folder, 'list_' + img_name + '.pt')
torch.save(patches_tensor, output_path)

print(f"Patches saved at {output_path}")

Patches saved at /content/drive/MyDrive/SealNN/Data/Maxar/list_22MAR25134903-P3DS-014983717010_01_P001.pt


## Classify the whole image and detect the seals